In [16]:
import os
import pandas as pd
import logging
import io
import requests
import threading
import queue
from tqdm import tqdm
import time
from bs4 import BeautifulSoup
from lxml import html

DATA_PATH = "/purchase/data"

In [17]:
os.listdir(DATA_PATH)

['features.csv',
 'INN_OUT_31_08_2021.xlsx',
 'out_file.csv',
 'res2.xlsx',
 'список инн.xlsx']

In [96]:
import aiohttp
import asynci

In [95]:
%%time

def get_web():
    print("Start")
    purch = "0819300026919000006"
    url="https://zakupki.gov.ru/epz/order/notice/ea44/view/supplier-results.html?regNumber="+purch
    r = requests.get(url, headers = headers)
    text = r.content.decode('utf-8')
    soup = BeautifulSoup(text)
    winer_list = soup.find_all('table',{'class': 'blockInfo__table tableBlock'})
    if len(winer_list) > 3:
        soup = BeautifulSoup(str(winer_list[3]))
    elif len(winer_list) == 3:
        soup = BeautifulSoup(str(winer_list[2]))
    else:
        return
    contract_link = soup.find('a')
    try:
        contract_url = contract_link.get('href')
    except Exception:
        return
    contract_url = 'https://zakupki.gov.ru' + contract_url
    try:
        contract_page = requests.get(contract_url, headers = headers)
    except Exception:
        return
    contract_page_text = contract_page.content.decode('utf-8')
    soup = BeautifulSoup(contract_page_text)
    spans = soup.find_all('span')
    ind_cur = -1
    print("Stop")
    for span in spans:
        if span.text == 'ИНН:': 
            ind_cur = spans.index(span)+1
            break
    
    if ind_cur == -1:
        return
    else:
        return spans[ind_cur].text

t = threading.Thread(target=get_web, args=())
t.start()
r = threading.Thread(target=get_web, args=())
r.start()
r.join()
t.join()



StartStart

Stop
Stop
CPU times: user 7.07 s, sys: 34.6 s, total: 41.6 s
Wall time: 14.7 s


In [81]:
headers = {'sec-ch-ua': '" Not;A Brand";v="99", "Google Chrome";v="91", "Chromium";v="91"',
           'sec-ch-ua-mobile':'0',
           'Upgrade-Insecure-Requests': '1',
           'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
                    }

def get_inn_vie_purch(purch):
    url="https://zakupki.gov.ru/epz/order/notice/ea44/view/supplier-results.html?regNumber="+purch
    r = requests.get(url, headers = headers)
    text = r.content.decode('utf-8')
    soup = BeautifulSoup(text)
    winer_list = soup.find_all('table',{'class': 'blockInfo__table tableBlock'})
    if len(winer_list) > 3:
        soup = BeautifulSoup(str(winer_list[3]))
    elif len(winer_list) == 3:
        soup = BeautifulSoup(str(winer_list[2]))
    else:
        return
    contract_link = soup.find('a')
    try:
        contract_url = contract_link.get('href')
    except Exception:
        return
    contract_url = 'https://zakupki.gov.ru' + contract_url
    try:
        contract_page = requests.get(contract_url, headers = headers)
    except Exception:
        return
    contract_page_text = contract_page.content.decode('utf-8')
    soup = BeautifulSoup(contract_page_text)
    spans = soup.find_all('span')
    ind_cur = -1
    for span in spans:
        if span.text == 'ИНН:': 
            ind_cur = spans.index(span)+1
            break
    if ind_cur == -1:
        return 
    else:
        return spans[ind_cur].text
tqdm.pandas()

In [82]:
def start_download(purch):
    out = purch
    scraping = threading.Thread(target=get_inn_vie_purch, args=(out,))
    scraping.start()
    scraping.join()
    while threading.active_count()>10:
        time.sleep(0.01)

In [83]:
df = pd.read_excel(os.path.join(DATA_PATH, 'INN_OUT_31_08_2021.xlsx'))

In [84]:
df.shape

(1488, 26)

In [85]:
df.head()

,Закупки по,Реестровый номер закупки,"Способ определения поставщика (подрядчика, исполнителя), подрядной организации (размещения закупки)",Наименование закупки,Предмет электронного аукциона (только для ПП РФ 615),Идентификационный код закупки,Номер лота,Наименование лота,Начальная (максимальная) цена контракта,Валюта,...,Наименование Заказчика,"Организация, осуществляющая размещение",Дата размещения,Дата обновления,Этап закупки,Особенности размещения заказа\ Преимущества (44-ФЗ),Дата начала подачи заявок,Дата окончания подачи заявок,Дата проведения электронного аукциона,INN
0,44-ФЗ,№0321100005821000031,Электронный аукцион,Поставка мебели,NaN,'211263507490826350100100230013101244',NaN,NaN,438658.00,RUB,...,УПРАВЛЕНИЕ ФЕДЕРАЛЬНОЙ СЛУЖБЫ ПО НАДЗОРУ В СФЕ...,УПРАВЛЕНИЕ ФЕДЕРАЛЬНОЙ СЛУЖБЫ ПО НАДЗОРУ В СФЕ...,19.08.2021,30.08.2021,Определение поставщика завершено,Организациям инвалидов в соответствии со ст. 2...,19.08.2021,27.08.2021,NaN,165711392901
1,44-ФЗ,№0136500001121004320,Электронный аукцион,Поставка мебели для офисов,NaN,'212690303967369500100100260040000244',NaN,NaN,472682.00,RUB,...,ГОСУДАРСТВЕННОЕ КАЗЕННОЕ УЧРЕЖДЕНИЕ ТВЕРСКОЙ О...,КОМИТЕТ ГОСУДАРСТВЕННОГО ЗАКАЗА ТВЕРСКОЙ ОБЛАСТИ,18.08.2021,30.08.2021,Определение поставщика завершено,NaN,18.08.2021,26.08.2021,NaN,165711392901
2,44-ФЗ,№0372200171121000069,Электронный аукцион,Поставка оборудования для обеспечения деятельн...,NaN,'212780600377978060100100490060000244',NaN,NaN,529284.00,RUB,...,САНКТ-ПЕТЕРБУРГСКОЕ ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ ...,САНКТ-ПЕТЕРБУРГСКОЕ ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ ...,18.08.2021,27.08.2021,Определение поставщика завершено,Организациям инвалидов в соответствии со ст. 2...,18.08.2021,26.08.2021,NaN,165711392901
3,44-ФЗ,№0848300048221000340,Электронный аукцион,"Приобретение мебели для радио-станции ""Танки-ФМ""",NaN,'213501705356050170100100070010000244',NaN,NaN,204699.98,RUB,...,"МУНИЦИПАЛЬНОЕ УЧРЕЖДЕНИЕ КУЛЬТУРЫ ""ЛЕНИНО-СНЕГ...","МУНИЦИПАЛЬНОЕ КАЗЕННОЕ УЧРЕЖДЕНИЕ ""ЦЕНТР ЗАКУПОК""",04.08.2021,27.08.2021,Определение поставщика завершено,Организациям инвалидов в соответствии со ст. 2...,04.08.2021,25.08.2021,NaN,165711392901
4,44-ФЗ,№0373200018021000010,Электронный аукцион,Электронный аукцион на поставку мебели для нуж...,NaN,'212771707173877170100100220010000244',NaN,NaN,2688612.12,RUB,...,ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ ПРОФЕССИОНАЛЬНОЕ ОБР...,ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ ПРОФЕССИОНАЛЬНОЕ ОБР...,16.08.2021,27.08.2021,Определение поставщика завершено,Организациям инвалидов в соответствии со ст. 2...,16.08.2021,24.08.2021,NaN,165711392901


In [86]:
df['purch'] = df['Реестровый номер закупки'].progress_apply(lambda x:x[1:])

100%|██████████| 1488/1488 [00:00<00:00, 397346.68it/s]


In [87]:
df['purch'].head(5)

0    0321100005821000031
1    0136500001121004320
2    0372200171121000069
3    0848300048221000340
4    0373200018021000010
Name: purch, dtype: object

In [69]:
df = df.loc[df['Закупки по']=='44-ФЗ']
df = df.loc[df['Этап закупки']=='Определение поставщика завершено']

In [70]:
#df = df[3000000:]

In [71]:
df.shape

(1327, 27)

In [74]:
print(start_download('0819300026919000007'))

2454028561


In [94]:
threads = []
for par in tqdm(df['purch'][:10]):
    new_thread = threading.Thread(target=start_download, args=(par,))
    new_thread.start()
    threads.append(new_thread)
    
for thread in threads:
    thread.join()

100%|██████████| 10/10 [00:00<00:00, 18.50it/s]


KeyboardInterrupt: 

In [91]:
for i in df['purch'][:5]:
    print(i)

0321100005821000031
0136500001121004320
0372200171121000069
0848300048221000340
0373200018021000010


In [88]:
df['winer'] = df['purch'].progress_apply(start_download)

  1%|▏         | 20/1488 [00:11<14:22,  1.70it/s]


KeyboardInterrupt: 

In [34]:
df.head(30)

,Закупки по,Реестровый номер закупки,"Способ определения поставщика (подрядчика, исполнителя), подрядной организации (размещения закупки)",Наименование закупки,Предмет электронного аукциона (только для ПП РФ 615),Идентификационный код закупки,Номер лота,Наименование лота,Начальная (максимальная) цена контракта,Валюта,...,Дата размещения,Дата обновления,Этап закупки,Особенности размещения заказа\ Преимущества (44-ФЗ),Дата начала подачи заявок,Дата окончания подачи заявок,Дата проведения электронного аукциона,INN,purch,winer
0,44-ФЗ,№0321100005821000031,Электронный аукцион,Поставка мебели,NaN,'211263507490826350100100230013101244',NaN,NaN,438658.00,RUB,...,19.08.2021,30.08.2021,Определение поставщика завершено,Организациям инвалидов в соответствии со ст. 2...,19.08.2021,27.08.2021,NaN,165711392901,0321100005821000031,0
1,44-ФЗ,№0136500001121004320,Электронный аукцион,Поставка мебели для офисов,NaN,'212690303967369500100100260040000244',NaN,NaN,472682.00,RUB,...,18.08.2021,30.08.2021,Определение поставщика завершено,NaN,18.08.2021,26.08.2021,NaN,165711392901,0136500001121004320,0
2,44-ФЗ,№0372200171121000069,Электронный аукцион,Поставка оборудования для обеспечения деятельн...,NaN,'212780600377978060100100490060000244',NaN,NaN,529284.00,RUB,...,18.08.2021,27.08.2021,Определение поставщика завершено,Организациям инвалидов в соответствии со ст. 2...,18.08.2021,26.08.2021,NaN,165711392901,0372200171121000069,0
3,44-ФЗ,№0848300048221000340,Электронный аукцион,"Приобретение мебели для радио-станции ""Танки-ФМ""",NaN,'213501705356050170100100070010000244',NaN,NaN,204699.98,RUB,...,04.08.2021,27.08.2021,Определение поставщика завершено,Организациям инвалидов в соответствии со ст. 2...,04.08.2021,25.08.2021,NaN,165711392901,0848300048221000340,0
4,44-ФЗ,№0373200018021000010,Электронный аукцион,Электронный аукцион на поставку мебели для нуж...,NaN,'212771707173877170100100220010000244',NaN,NaN,2688612.12,RUB,...,16.08.2021,27.08.2021,Определение поставщика завершено,Организациям инвалидов в соответствии со ст. 2...,16.08.2021,24.08.2021,NaN,165711392901,0373200018021000010,0
5,44-ФЗ,№0373200046921000031,Запрос котировок в электронной форме,Поставка мебели (Закупка 1),NaN,'212500300928077510100100780010000244',NaN,NaN,153000.00,RUB,...,18.08.2021,26.08.2021,Определение поставщика завершено,Организациям инвалидов в соответствии со ст. 2...,18.08.2021,25.08.2021,NaN,165711392901,0373200046921000031,710503069780
6,44-ФЗ,№0801300012821000103,Электронный аукцион,Поставка мебели,NaN,'213024502724002450100100300030000244',NaN,NaN,673019.98,RUB,...,06.08.2021,24.08.2021,Определение поставщика завершено,Организациям инвалидов в соответствии со ст. 2...,06.08.2021,20.08.2021,NaN,165711392901,0801300012821000103,0
7,44-ФЗ,№0848300046121000266,Запрос котировок в электронной форме,Поставка мебели,NaN,'213502802326150280100100210010000000',NaN,NaN,345615.90,RUB,...,10.08.2021,19.08.2021,Определение поставщика завершено,Организациям инвалидов в соответствии со ст. 2...,10.08.2021,18.08.2021,NaN,165711392901,0848300046121000266,7716748907
8,44-ФЗ,№0373200032321000002,Электронный аукцион,"Поставка мебели для нужд ГБПОУ ДЗМ ""МК №6"" в 2...",NaN,'212771603428277160100100150010000244',NaN,NaN,370348.68,RUB,...,09.08.2021,18.08.2021,Определение поставщика завершено,Организациям инвалидов в соответствии со ст. 2...,09.08.2021,17.08.2021,NaN,165711392901,0373200032321000002,165711392901
9,44-ФЗ,№0148300028021000014,Запрос котировок в электронной форме,Поставка мебели для нужд администрации поселен...,NaN,'213507403203877510100100340140000244',NaN,NaN,523332.02,RUB,...,10.08.2021,18.08.2021,Определение поставщика завершено,Организациям инвалидов в соответствии со ст. 2...,10.08.2021,17.08.2021,NaN,165711392901,0148300028021000014,7716748907


In [35]:
df.to_excel('res2.xlsx')